In [5]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import fivecentplots as fcp
import janitor
import typing

In [6]:
df = pd.read_csv("assign2_wastedata.csv")

DATE='Date'
BUILDING='Building'
WASTE_TYPE='Waste Type'
SUBSTREAM='Substream'
DISPOSED_IN='Disposed In'
VOL='Volume'
WEIGHT='Weight'
NOTES='Notes'

CLR_REC_BLD='#451AFF'
CLR_LFILL_BLD='#828282'
CLR_CMP_BLD='#00B600'
CLR_REU_BLD='#ffa500'
CLR_REU_MIL='#ffe148'
CLR_REC_MIL='#4A62FF'
CLR_LFILL_MIL='#AFAFAF'
CLR_CMP_MIL='#7AE288'

clr_map_bld = {
    'Recycling': CLR_REC_BLD,
    'Reusables': CLR_REU_BLD,
    'Compost': CLR_CMP_BLD,
    'Landfill': CLR_LFILL_BLD
}

clr_map_mld = {
    'Recycling': CLR_REC_MIL,
    'Reusables': CLR_REU_MIL,
    'Compost': CLR_CMP_MIL,
    'Landfill': CLR_LFILL_MIL
}

## ----- Pre-processing and cleaning steps --------

# --- Separating Stream into Waste Type and Disposed In
def split_stream(stream: str):
    resp = stream.split(" in ")
    if len(resp) == 1:
        return resp[0].strip(), resp[0].strip()
    return resp[0].strip(), resp[1].strip()

df = df.join(pd.DataFrame(df['Stream'].apply(split_stream).tolist(), columns=["Waste Type", "Disposed In"])).drop('Stream', axis=1)
df = df[[DATE, BUILDING, WASTE_TYPE, SUBSTREAM, DISPOSED_IN,  VOL, WEIGHT, NOTES]]

#  --- Renaming WasteType==Food Waste to Compost. ---
# Since Waste_typw==Food Waste has a miniscule amount (only has 8.1 pounds of total data, i.e. 0.2%)
# we're not loosing much information.
# it is safe to club as Compost waste.
df[WASTE_TYPE] = df[WASTE_TYPE].str.replace('Food Waste', 'Compost', regex=False)


# --- Clubbing Substreams where-ever possible ---
df['Substream'] = df['Substream'].str.capitalize()
# rename str mapper
rename_str = {
    'Aseptic containers': ['Aseptic/tetra-pak'],
    'Electronic & universal waste': ['E-/universal waste'],
    'Plastic 1-7': ['Film plastic', 'Plastics #1-7', 'Plastic film'],
    'Food/organic waste': ['Food waste (edible)'],
    'Metal': ['Metal & aluminum'],
    'Paperboard': ['Paperboard rolls'],
}

for key, val in rename_str.items():
    for i in val:
        df[SUBSTREAM] = df[SUBSTREAM].str.replace(i, key, regex=False)

df

,Date,Building,Waste Type,Substream,Disposed In,Volume,Weight,Notes
0,5/15/18,Facilities,Recycling,Paper,Recycling,1.00,15.8,NaN
1,5/15/18,Facilities,Recycling,Cardboard,Recycling,1.00,5.5,NaN
2,5/15/18,Facilities,Recycling,Cardboard,Recycling,1.00,8.6,NaN
3,5/15/18,Facilities,Recycling,Paper,Recycling,0.66,6.5,NaN
4,5/15/18,Facilities,Landfill,Other landfill,Recycling,0.10,1.2,coffee cups
...,...,...,...,...,...,...,...,...
516,3/14/23,Swig,Recycling,Plastic 1-7,Landfill,1.00,6.4,plastic bottles; plastic food containers
517,3/14/23,Swig,Recycling,Plastic 1-7,Landfill,1.00,4.9,bottles
518,3/14/23,Swig,Reusables,Reusables,Landfill,0.25,4.9,clothing
519,3/14/23,Swig,Landfill,Styrofoam,Landfill,0.00,0.0,5 pieces of styrofoam


In [8]:
def _stylize_words(tempdf, col_name, emoji: bool=False) -> pd.DataFrame:
    """
    Returns df containing stylized colname
    """
    col_name_style = f'{col_name}_style'
    tempdf[col_name_style] = tempdf[col_name].copy()
    if emoji:
        tempdf[col_name_style] = tempdf[col_name_style].str.replace('Compost', f"<b style='color:{clr_map_bld['Compost']};'>Compost</b> <b>🍃</b>")
        tempdf[col_name_style] = tempdf[col_name_style].str.replace('Landfill', 'Landfill 🕳️')
        tempdf[col_name_style] = tempdf[col_name_style].str.replace('Recycling', 'Recycling ♻️')
        tempdf[col_name_style] = tempdf[col_name_style].str.replace('Reusable', 'Reusable 🔃')
    else:
        tempdf[col_name_style] = tempdf[col_name_style].str.replace('Compost', f"<b style='color:{clr_map_bld['Compost']};'>Compost</b> <b>🍃</b>")
        tempdf[col_name_style] = tempdf[col_name_style].str.replace('Landfill', f"<b style='color:{clr_map_bld['Landfill']};'>Landfill</b> 🕳️")
        tempdf[col_name_style] = tempdf[col_name_style].str.replace('Recycling', f"<b style='color:{clr_map_bld['Recycling']};'>Recycling</b> ♼")
        tempdf[col_name_style] = tempdf[col_name_style].str.replace('Reusable', f"<b style='color:{clr_map_bld['Reusables']};'>Reusables</b> ♲")

    return df, col_name_style

## Graphs to plot:
- For eacho of the points, you'll have 2 visualizations: one for the total and another one for each building
0. A bird's eye view. How each building is doing, how much waste is collected, total volume, weight, timeline etc
1. Sunbun graph of wastes and substreams
2. How correctly is SCU disposing its waste
   - Sub categories can be based on the substream
   1. Graph showing misclassified and classified waste types. And how often are they misclassified. 
   2. This can be by %age and by volume

**Experiment idea**
- Find the square foot/occupancy of each of the plots. Calculate waste generated per sq unit or per 100 people etc.

## 0. Bird's eye view

## 1. Sunburn of waste and substreams

In [9]:
# fig = px.sunburst(df, path=[BUILDING, WASTE_TYPE, SUBSTREAM], values=WEIGHT)
fig = px.treemap(df, path=[BUILDING, WASTE_TYPE, SUBSTREAM], values=WEIGHT, color_discrete_map=clr_map_mld, color=WASTE_TYPE)
fig.update_layout(title='Waste Characterization of individual building at SCU')
fig.show()

## 2. How 'Correctly' is SCU disposing its waste

### 2: Waste misclassification based on total volume

#### **First things first. How much waste is being misclassified?**

In [10]:
def get_misclassif_perc(level: str, building: str = None, df: pd.DataFrame = df):
    """
    level: waste_type: columns contain waste_type, weight and %age
            substream: contains waste_type, substream, weight and %age
            disposed_in: contains waste_type, substream, disposed_in, weight and %page
    building: Building 
    """
    temp_df = df.loc[:] if not building else df.loc[df[BUILDING]==building, :]
    TOTAL_DISCLASSIFIED_WEIGHT = temp_df.loc[temp_df[WASTE_TYPE]!=temp_df[DISPOSED_IN]][WEIGHT].sum()
    
    if level=='waste_type':
        wgt = temp_df.loc[temp_df[WASTE_TYPE]!=temp_df[DISPOSED_IN]].groupby([WASTE_TYPE]).agg({WEIGHT: 'sum'})
        wgt['%'] = wgt[WEIGHT] / TOTAL_DISCLASSIFIED_WEIGHT * 100
    elif level=='substream':
        wgt = temp_df.loc[temp_df[WASTE_TYPE]!=temp_df[DISPOSED_IN]].groupby([WASTE_TYPE, SUBSTREAM]).agg({WEIGHT: 'sum'})#.reset_index()
        wgt['%'] = wgt[WEIGHT] / wgt.groupby([WASTE_TYPE])[WEIGHT].transform('sum') * 100
    elif level=='disposed_in':
        wgt = temp_df.loc[temp_df[WASTE_TYPE]!=temp_df[DISPOSED_IN]].groupby([WASTE_TYPE, SUBSTREAM, DISPOSED_IN]).agg({WEIGHT: 'sum'})
        wgt['%'] =  wgt[WEIGHT] / wgt.groupby([WASTE_TYPE, SUBSTREAM])[WEIGHT].transform('sum') * 100
    return wgt

# op = get_misclassif_perc('substream')
op = get_misclassif_perc('disposed_in')
op#[WEIGHT]#[:13].sum()
op.index.unique()

MultiIndex([(  'Compost',  'Bpi certified compostable to-go ware', ...),
            (  'Compost', 'Compostable dining waste (non-edible)', ...),
            (  'Compost', 'Compostable dining waste (non-edible)', ...),
            (  'Compost',           'Compostable food containers', ...),
            (  'Compost',           'Compostable food containers', ...),
            (  'Compost',                    'Food/organic waste', ...),
            (  'Compost',                    'Food/organic waste', ...),
            (  'Compost',                      'Misc. meal waste', ...),
            (  'Compost',                      'Misc. meal waste', ...),
            (  'Compost',                  'Napkins/paper towels', ...),
            (  'Compost',                          'Paper towels', ...),
            (  'Compost',                          'Paper towels', ...),
            (  'Compost',                       'Scu coffee cups', ...),
            ( 'Landfill',                        'O

In [11]:
# op.groupby(level=0).apply(lambda op: op.xs(op.name).to_dict()).to_dict()
# op.index.get_value
tdf1 = op.reset_index()
wt_substream_mapper = {wt: tdf1.loc[tdf1[WASTE_TYPE]==wt, :][SUBSTREAM].unique().tolist() for wt in tdf1[WASTE_TYPE].unique()}
# wts
tmp = list(map(lambda x: f'{clr_map_mld["Compost"]}: {x}', wt_substream_mapper["Compost"]))
tmp


['#7AE288: Bpi certified compostable to-go ware',
 '#7AE288: Compostable dining waste (non-edible)',
 '#7AE288: Compostable food containers',
 '#7AE288: Food/organic waste',
 '#7AE288: Misc. meal waste',
 '#7AE288: Napkins/paper towels',
 '#7AE288: Paper towels',
 '#7AE288: Scu coffee cups']

In [12]:
import numpy as np
def _plot_disposed_in_graph(df: pd.DataFrame, keys):
    """
    df should be a multi-indexed df having (WASTETYPE, SUBSTREAM, DISPOSED) as the index
    keys should be a list of tuples. Each tuple has 2 element representing: (WasteType, Substream).
    It represents which waste types to display on the grpah
    """
    wtdf: pd.DataFrame = df.select_rows(keys)
    wtdf = wtdf.reset_index()
    fig = make_subplots(rows=1, cols=1)

    fig.add_trace(
        go.Bar(
            y=[wtdf[WASTE_TYPE].tolist(), wtdf[SUBSTREAM].tolist(), wtdf[DISPOSED_IN].tolist()],
            x=wtdf[WEIGHT],
            customdata=wtdf[DISPOSED_IN].tolist(),# wtdf['%'].tolist()]),
            orientation='h',
            marker={'color': [clr_map_mld[clr] for clr in wtdf[DISPOSED_IN].tolist()]},
            hovertext=wtdf[DISPOSED_IN].tolist(),
            hovertemplate=
            "<b>Disposed In</b><br><br>%{customdata}: %{x} pound(s)"
        )
    )

    # title = f'Different of types of waste wrongly thrown into the <b style="color:{clr_map_bld[bin_name]};">{bin_name}</b> bin '
    # title+= f'for <b>{building_name}</b> building' if building_name else ''
    fig.update_xaxes(title_text='Weight (in pounds)')
    fig.update_yaxes(title_text='Waste Types')
    fig.update_layout(title="Waste Types, Substreams and how they're <b>Wrongly</b> Disposed In different bins")
    return fig, wtdf


fig, tdf = _plot_disposed_in_graph(op, [('Compost', 'Paper towels'), ('Compost', 'Misc. meal waste'), ('Landfill', 'Reusables'), ('Recycling', 'Cardboard'), ('Reusables', 'Reusables')])
fig.show()

tdf


,Waste Type,Substream,Disposed In,Weight,%
0,Compost,Paper towels,Landfill,205.10,91.644325
1,Compost,Paper towels,Recycling,18.70,8.355675
2,Compost,Misc. meal waste,Landfill,762.70,99.090555
3,Compost,Misc. meal waste,Recycling,7.00,0.909445
4,Landfill,Reusables,Recycling,2.00,100.000000
5,Recycling,Cardboard,Compost,0.80,0.840336
6,Recycling,Cardboard,Landfill,94.40,99.159664
7,Reusables,Reusables,Compost,1.21,0.944501
8,Reusables,Reusables,Landfill,103.10,80.477714
9,Reusables,Reusables,Recycling,23.80,18.577785


#### **What kind of waste is most oftenly misclassified?**
- Interms of volume and %age

In [13]:
temp_df = df.loc[:]
# interms of weight
wgt = temp_df.loc[temp_df[WASTE_TYPE]!=temp_df[DISPOSED_IN]].groupby([WASTE_TYPE]).agg({WEIGHT: 'sum'}).sort_values(WEIGHT, ascending=False)

# interms of %age
perc = temp_df.loc[temp_df[WASTE_TYPE]!=temp_df[DISPOSED_IN]].groupby([WASTE_TYPE]).agg({WEIGHT: 'sum'}) / TOTAL_DISCLASSIFIED_WEIGHT * 100
perc = perc.sort_values(WEIGHT, ascending=False)
perc

NameError: name 'TOTAL_DISCLASSIFIED_WEIGHT' is not defined

#### **So compost is oftenly misclassified. But where are we going wrong?**
- Interms of volume and %age

In [ ]:
temp_df = df.loc[:]
# interms of weight
wgt = temp_df.loc[temp_df[WASTE_TYPE]!=temp_df[DISPOSED_IN]].groupby([WASTE_TYPE, SUBSTREAM]).agg({WEIGHT: 'sum'})#.sort_values(WEIGHT, ascending=False)
wgt #.sort_values(WEIGHT, ascending=False)

## interms of %age
perc = temp_df.loc[temp_df[WASTE_TYPE]!=temp_df[DISPOSED_IN]].groupby([WASTE_TYPE, SUBSTREAM]).agg({WEIGHT: 'sum'}) / TOTAL_DISCLASSIFIED_WEIGHT * 100
perc.sort_values(WEIGHT, ascending=False)

Weight
Waste Type Substream                                       
Compost    Misc. meal waste                       31.021655
Recycling  Plastic 1-7                            16.331408
Compost    Compostable food containers            11.252756
           Paper towels                            9.019938
Recycling  Paper                                   6.074151
Reusables  Reusables                               5.163290
Recycling  Glass                                   4.090812
           Cardboard                               3.836900
Compost    Food/organic waste                      3.587017
Recycling  Metal                                   2.185259
Landfill   Other landfill                          1.423522
Compost    Compostable dining waste (non-edible)   0.999528
Recycling  Const/demo waste                        0.957210
           Terracycle                              0.873378
           Paperboard                              0.687579
           Aseptic containers                      0.644857
           Electronic & universal waste            0.464297
Compost    Scu coffee cups                         0.415127
           Bpi certified compostable to-go ware    0.306307
           Napkins/paper towels                    0.286155
Recycling  Paper towels                            0.274064
Landfill   Reusables                               0.080607
           Styrofoam                               0.024182

#### **Okay so we know what substreams are most oftenly misclassified. But with what bin are we misclassifying it?**

In [14]:
temp_df = df.loc[:]
# interms of weight
wgt = temp_df.loc[temp_df[WASTE_TYPE]!=temp_df[DISPOSED_IN]].groupby([WASTE_TYPE, SUBSTREAM, DISPOSED_IN]).agg({WEIGHT: 'sum'})#.sort_values(WEIGHT, ascending=False)
wgt#.sort_values(WEIGHT, ascending=False)

## %age
wgt['%'] = 100 * wgt[WEIGHT] / wgt.groupby([WASTE_TYPE, SUBSTREAM])[WEIGHT].transform('sum')
wgt#.reset_index().sort_values([WEIGHT, WASTE_TYPE,SUBSTREAM], ascending=False)

Weight   
Waste Type Substream                             Disposed In           
Compost    Bpi certified compostable to-go ware  Landfill       7.60  \
           Compostable dining waste (non-edible) Landfill      24.20   
                                                 Recycling      0.60   
           Compostable food containers           Landfill     267.70   
                                                 Recycling     11.50   
           Food/organic waste                    Landfill      87.30   
                                                 Recycling      1.70   
           Misc. meal waste                      Landfill     762.70   
                                                 Recycling      7.00   
           Napkins/paper towels                  Landfill       7.10   
           Paper towels                          Landfill     205.10   
                                                 Recycling     18.70   
           Scu coffee cups                       Landfill      10.30   
Landfill   Other landfill                        Compost        4.81   
                                                 Recycling     30.51   
           Reusables                             Recycling      2.00   
           Styrofoam                             Recycling      0.60   
Recycling  Aseptic containers                    Compost        0.60   
                                                 Landfill      15.40   
           Cardboard                             Compost        0.80   
                                                 Landfill      94.40   
           Const/demo waste                      Landfill      23.75   
           Electronic & universal waste          Compost        0.01   
                                                 Landfill      11.51   
           Glass                                 Compost        1.90   
                                                 Landfill      99.60   
           Metal                                 Compost        1.00   
                                                 Landfill      53.22   
           Paper                                 Compost        1.01   
                                                 Landfill     149.70   
           Paper towels                          Landfill       6.80   
           Paperboard                            Compost        0.75   
                                                 Landfill      16.31   
           Plastic 1-7                           Compost        7.01   
                                                 Landfill     398.20   
           Terracycle                            Compost        1.90   
                                                 Landfill      19.77   
Reusables  Reusables                             Compost        1.21   
                                                 Landfill     103.10   
                                                 Recycling     23.80   

                                                                       %  
Waste Type Substream                             Disposed In              
Compost    Bpi certified compostable to-go ware  Landfill     100.000000  
           Compostable dining waste (non-edible) Landfill      97.580645  
                                                 Recycling      2.419355  
           Compostable food containers           Landfill      95.881089  
                                                 Recycling      4.118911  
           Food/organic waste                    Landfill      98.089888  
                                                 Recycling      1.910112  
           Misc. meal waste                      Landfill      99.090555  
                                                 Recycling      0.909445  
           Napkins/paper towels                  Landfill     100.000000  
           Paper towels                          Landfill      91.644325  
                                                 Recycling    

### 2.a. Misclassified and classified waste types

In [27]:
def get_wrong_waste_agg_df(bin_name: str, building_name: str = None):
    if not building_name:
        group_df = df.loc[df[WASTE_TYPE]!=df[DISPOSED_IN], :].groupby([DISPOSED_IN,WASTE_TYPE, SUBSTREAM]).agg({WEIGHT: 'sum'})[WEIGHT].reset_index()
    else:
        group_df = df.loc[(df[WASTE_TYPE]!=df[DISPOSED_IN]) & (df[BUILDING]==building_name), :].groupby([DISPOSED_IN,WASTE_TYPE, SUBSTREAM]).agg({WEIGHT: 'sum'})[WEIGHT].reset_index()
    group_df
    wtdf = group_df.loc[group_df[DISPOSED_IN]==bin_name] # waste type df
    return wtdf


def plot_waste_division(bin_name: str, building_name: str = None):
    wtdf = get_wrong_waste_agg_df(bin_name, building_name)
    per_bin_max_elements=5

    # ------ optional code to select only top n rows within each group -------
    # idx2keep = []
    # for wt in wtdf[WASTE_TYPE].unique():
    #     idx2keep.extend(wtdf.loc[wtdf[WASTE_TYPE]==wt, :].sort_values(WEIGHT, ascending=False)[:per_bin_max_elements].index.tolist())

    # wtdf = wtdf.loc[idx2keep, :]
    # ----------------------------------------------------------------------

    fig = make_subplots(rows=1, cols=1)

    fig.add_trace(
        go.Bar(
            y=[wtdf[WASTE_TYPE].tolist(), wtdf[SUBSTREAM].tolist()],
            x=wtdf[WEIGHT],
            orientation='h',
            marker={'color': [clr_map_mld[clr] for clr in wtdf[WASTE_TYPE].tolist()]},
        )
    )

    title = f'Different of types of waste wrongly thrown into the <b style="color:{clr_map_bld[bin_name]};">{bin_name}</b> bin '
    title+= f'for <b>{building_name}</b> building' if building_name else ''
    fig.update_xaxes(title_text='Weight (in pounds)')
    fig.update_yaxes(title_text='Waste Type')
    fig.update_layout(title=title)

    return fig

bldn = ['Facilities', 'Swig', 'Vari Hall and Lucas Hall', 'Malley',
       'University Villas', 'Graham', 'Benson Center', 'Learning Commons']


bd = bldn[7]
fig = plot_waste_division('Recycling', bd)#, 'Facilities')
fig.show()

fig = plot_waste_division('Landfill', bd)#, 'Facilities')
fig.show()

fig = plot_waste_division('Compost', bd)#, 'Facilities')
fig.show()

In [22]:
df.loc[df[BUILDING]==bd, DISPOSED_IN].unique()

array(['Landfill'], dtype=object)